## Доказательство эффективности самописного beam-search + добавление LM в beam-search

In [ ]:
# Клонируем репозиторий
!git clone https://github.com/ZenMan123/ASR_HM1.git

In [ ]:
# Ставим все необходимые библиотеки
!pip install -r ASR_HM1/requirements.txt

In [ ]:
# Подгружаем финальную модельку

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="ZenMan67/asr_checkpoint_baseline",
    repo_type="model",
    local_dir="ASR_HM1/saved/asr_checkpoint_baseline",
    local_dir_use_symlinks=False
)

### Самописный beam-search

In [ ]:
# Переходим в нужную папочку и запускаем замер
# Тут поставил (limit=500), потому что работает самописный бимсерч очень долго (на 500 сэмплах около часа)

!(cd ASR_HM1 && python inference.py metrics=custom_beam datasets=test_only \
    inferencer.from_pretrained="saved/asr_checkpoint_baseline/model_best.pth" \
    text_encoder=ctc_text_encoder)

Сравнивать результаты с [этим](https://www.comet.com/artemsafin67/pytorch-template-asr-example/me9ph890x936gkyhjxyh9ohzgo8a461q?compareXAxis=step&experiment-tab=panels&prevPath=%2Fartemsafin67%2Fpytorch-template-asr-example%2Fview%2Fnew%2Fpanels&showOutliers=true&smoothing=0&xAxis=step) замером. В нем для инференса использовался beam_search из библиотеки `pyctcdecode`. Видно, что мой beam search позволяет выбить приблизительно такое же качество. 

### Добавление LM в beam-search

In [ ]:
!(cd ASR_HM1 && python inference.py metrics=beam_lm datasets=eval \
    inferencer.from_pretrained="saved/asr_checkpoint_baseline/model_best.pth" \
    text_encoder=ctc_text_encoder)

Сравнивать результаты с [этим](https://www.comet.com/artemsafin67/pytorch-template-asr-example/me9ph890x936gkyhjxyh9ohzgo8a461q?compareXAxis=step&experiment-tab=panels&prevPath=%2Fartemsafin67%2Fpytorch-template-asr-example%2Fview%2Fnew%2Fpanels&showOutliers=true&smoothing=0&xAxis=step) замером. В нем для инференса использовался beam_search без встроенной LM. Видно, что beam search + LM выбивает результат лучше (WER упал с 0.46 до 0.37)